In [13]:
with open('file.txt', 'r') as file:
    lines = file.read().strip().split('\n')

# Example data from problem description
example_lines = [
    "[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}",
    "[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}",
    "[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}"
]

print("Example data:")
for line in example_lines:
    print(line)

print(f"\nReal data: {len(lines)} machines")

Example data:
[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}
[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}
[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}

Real data: 195 machines


In [14]:
# === Parse the machine specifications ===

import re

def parse_machine(line):
    """Parse a machine line into target state and buttons"""
    # Extract indicator lights pattern [.##.]
    lights_match = re.search(r'\[(.*?)\]', line)
    lights_str = lights_match.group(1)
    target = [1 if c == '#' else 0 for c in lights_str]
    
    # Extract all button specifications (0,1,2)
    buttons = []
    button_matches = re.findall(r'\(([0-9,]+)\)', line)
    for button_str in button_matches:
        indices = [int(x) for x in button_str.split(',')]
        buttons.append(indices)
    
    return target, buttons

# Test parsing
print("Testing parser:")
for i, line in enumerate(example_lines):
    target, buttons = parse_machine(line)
    print(f"\nMachine {i+1}:")
    print(f"  Target: {target} ({len(target)} lights)")
    print(f"  Buttons: {buttons} ({len(buttons)} buttons)")

Testing parser:

Machine 1:
  Target: [0, 1, 1, 0] (4 lights)
  Buttons: [[3], [1, 3], [2], [2, 3], [0, 2], [0, 1]] (6 buttons)

Machine 2:
  Target: [0, 0, 0, 1, 0] (5 lights)
  Buttons: [[0, 2, 3, 4], [2, 3], [0, 4], [0, 1, 2], [1, 2, 3, 4]] (5 buttons)

Machine 3:
  Target: [0, 1, 1, 1, 0, 1] (6 lights)
  Buttons: [[0, 1, 2, 3, 4], [0, 3, 4], [0, 1, 2, 4, 5], [1, 2]] (4 buttons)


In [19]:
# === Gaussian Elimination over GF(2) with Minimum Solution ===

import numpy as np
from itertools import product

def solve_gf2(target, buttons):
    """
    Solve the system of linear equations over GF(2) to find MINIMUM button presses.
    
    We want to find x (button presses) such that:
    A * x = target (mod 2)
    
    where A is a matrix where A[i][j] = 1 if button j toggles light i
    """
    n_lights = len(target)
    n_buttons = len(buttons)
    
    # Create the augmented matrix [A | target]
    matrix = []
    for light_idx in range(n_lights):
        row = []
        for button in buttons:
            row.append(1 if light_idx in button else 0)
        row.append(target[light_idx])
        matrix.append(row)
    
    matrix = np.array(matrix, dtype=int)
    
    # Reduced Row Echelon Form (RREF) in GF(2)
    rows, cols = matrix.shape
    pivot_cols = []
    current_row = 0
    
    for col in range(n_buttons):
        # Find pivot
        pivot_row = None
        for row in range(current_row, rows):
            if matrix[row, col] == 1:
                pivot_row = row
                break
        
        if pivot_row is None:
            continue
        
        # Swap rows
        if pivot_row != current_row:
            matrix[[current_row, pivot_row]] = matrix[[pivot_row, current_row]]
        
        pivot_cols.append(col)
        
        # Eliminate all other 1s in this column (forward AND backward)
        for row in range(rows):
            if row != current_row and matrix[row, col] == 1:
                matrix[row] = (matrix[row] + matrix[current_row]) % 2
        
        current_row += 1
    
    # Check for inconsistency
    for row in range(current_row, rows):
        if matrix[row, -1] == 1:
            return None  # No solution
    
    # Identify free variables (non-pivot columns)
    free_vars = [col for col in range(n_buttons) if col not in pivot_cols]
    
    # If no free variables, there's a unique solution
    if not free_vars:
        solution = [0] * n_buttons
        for row, col in enumerate(pivot_cols):
            solution[col] = matrix[row, -1]
        return solution
    
    # Try all combinations of free variables to find minimum solution
    min_presses = float('inf')
    best_solution = None
    
    for free_values in product([0, 1], repeat=len(free_vars)):
        solution = [0] * n_buttons
        
        # Set free variables
        for i, col in enumerate(free_vars):
            solution[col] = free_values[i]
        
        # Calculate pivot variables based on free variables
        for row, col in enumerate(pivot_cols):
            val = matrix[row, -1]
            for j in range(col + 1, n_buttons):
                val = (val + matrix[row, j] * solution[j]) % 2
            solution[col] = val
        
        # Count total presses
        presses = sum(solution)
        if presses < min_presses:
            min_presses = presses
            best_solution = solution
    
    return best_solution

# Test with examples
print("Testing Gaussian Elimination:")
for i, line in enumerate(example_lines):
    target, buttons = parse_machine(line)
    solution = solve_gf2(target, buttons)
    
    if solution:
        total_presses = sum(solution)
        print(f"\nMachine {i+1}: {total_presses} presses")
        print(f"  Solution: {solution}")
        
        # Verify the solution
        result = [0] * len(target)
        for btn_idx, presses in enumerate(solution):
            if presses == 1:
                for light in buttons[btn_idx]:
                    result[light] = (result[light] + 1) % 2
        print(f"  Target:   {target}")
        print(f"  Result:   {result}")
        print(f"  Valid: {result == target}")
    else:
        print(f"\nMachine {i+1}: No solution!")

Testing Gaussian Elimination:

Machine 1: 2 presses
  Solution: [np.int64(0), np.int64(0), np.int64(0), 0, np.int64(1), 1]
  Target:   [0, 1, 1, 0]
  Result:   [0, 1, 1, 0]
  Valid: True

Machine 2: 3 presses
  Solution: [np.int64(0), np.int64(0), 1, np.int64(1), np.int64(1)]
  Target:   [0, 0, 0, 1, 0]
  Result:   [0, 0, 0, 1, 0]
  Valid: True

Machine 3: 2 presses
  Solution: [np.int64(0), np.int64(1), np.int64(1), 0]
  Target:   [0, 1, 1, 1, 0, 1]
  Result:   [0, 1, 1, 1, 0, 1]
  Valid: True


In [20]:
# === Solve for all machines ===

total_presses = 0

print("Solving all machines...\n")

for i, line in enumerate(lines):
    target, buttons = parse_machine(line)
    solution = solve_gf2(target, buttons)
    
    if solution:
        presses = sum(solution)
        total_presses += presses
        
        if i < 5:  # Show first 5
            print(f"Machine {i+1}: {presses} presses")
    else:
        print(f"Machine {i+1}: NO SOLUTION!")

print(f"\n{'='*60}")
print(f"ANSWER: Total button presses = {total_presses}")
print(f"{'='*60}")

Solving all machines...

Machine 1: 3 presses
Machine 2: 1 presses
Machine 3: 1 presses
Machine 4: 3 presses
Machine 5: 6 presses

ANSWER: Total button presses = 538


In [21]:
# === Verify with example (expected: 7) ===

total = 0

for i, line in enumerate(example_lines):
    target, buttons = parse_machine(line)
    solution = solve_gf2(target, buttons)
    presses = sum(solution)
    total += presses
    print(f"Machine {i+1}: {presses} presses")

print(f"\nTotal: {total}, expected: 7")



Machine 1: 2 presses
Machine 2: 3 presses
Machine 3: 2 presses

Total: 7, expected: 7
